In [1]:
import sys
sys.path.append('/home/zhx/word/work/DriverOrderOfflineRL/cage-challenge-1/CybORG')
sys.path.append('/home/zhx/word/work/DriverOrderOfflineRL/cyborg-env')

In [2]:
import gymnasium as gym
import cyborg_env

In [3]:
import ray
import numpy as np
import torch
import gymnasium as gym
from gymnasium import spaces
from gymnasium.wrappers import EnvCompatibility
from ray.tune.registry import register_env

In [4]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Shared.Actions import *
from CybORG.Agents import RedMeanderAgent, B_lineAgent
from CybORG.Agents.Wrappers import *

# seed
np.random.seed(1)
torch.manual_seed(1)

env = gym.make('cyborg')

/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/gymnasium/wrappers/compatibility.py:60: DeprecationWarning: WARN: The `gymnasium.make(..., apply_api_compatibility=...)` parameter is deprecated and will be removed in v1.0. Instead use `gymnasium.make('GymV21Environment-v0', env_name=...)` or `from shimmy import GymV21CompatibilityV0`
  logger.deprecation(


In [4]:
from ipywidgets import Output
from IPython import display
import time

# The following three lines are for rendering purposes only.
# They allow us to render the env frame-by-frame in-place
# (w/o creating a huge output which we would then have to scroll through).
out = Output()
display.display(out)
with out:

    # Putting the Gym simple API methods together.
    # Here is a pattern for running a bunch of episodes.
    num_episodes = 5 # Number of episodes you want to run the agent
    total_reward = 0.0  # Initialize reward to 0

    # Loop through episodes
    for ep in range(num_episodes):

        # Reset the environment at the start of each episode
        obs, _ = env.reset()
        done = False

        # Loop through time steps per episode
        for i in range(100):
            # take random action, but you can also do something more intelligent 
            action = env.action_space.sample()

            # apply the action
            new_obs, reward, done, _, info = env.step(action)
            total_reward += reward

            # If the epsiode is up, then start another one
            if done:
                break

            # Render the env (in place).
            out.clear_output(wait=True)
            print(f"episode: {ep}")
            print(f"obs: {new_obs}, reward: {total_reward}, done: {done}")

Output()

In [8]:
# import commonly-used libraries
import os
import time
import numpy as np
print(f'Number of CPUs in this system: {os.cpu_count()}')
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
print(f"numpy: {np.__version__}")
print(f"pandas: {pd.__version__}")

# import ray
import ray
from ray import tune, rllib, air
from ray.tune.logger import pretty_print
print(f"ray: {ray.__version__}")

Number of CPUs in this system: 72
numpy: 1.24.3
pandas: 2.0.3


2023-12-02 11:24:12,835	WARNING compression.py:16 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


ray: 2.8.0


In [5]:
from ray.rllib.utils.pre_checks.env import check_env

# How to check you do not have any environment errors
print("checking environment ...")
try:
    check_env(env)
    print("All checks passed. No errors found.")
except:
    print("failed")

2023-12-02 12:02:04,472	WARNING compression.py:16 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


checking environment ...
All checks passed. No errors found.


In [6]:
from ray.tune.registry import register_env
from cyborg_env.envs.cyborg_env import CyborgEnv

In [7]:
env_creator = lambda config: CyborgEnv()
# register that way to make the environment under an rllib name
register_env('cyborg', lambda config: env_creator(config))

In [8]:
# config is an object instead of a dictionary since Ray version >= 1.13
from ray.rllib.algorithms.dqn import DQNConfig

# Default DQN config values
# uncomment below to see the long list of specifically PPO default config values
# print(pretty_print(DQNConfig().to_dict()))

In [9]:
# Create a DQNConfig object
dqn_config = DQNConfig()

# Only for Colab. Specify 1 gpu
# dqn_config.num_gpus=1

# Setup our config object to use our environment
dqn_config.environment(env='cyborg')

# Decide if you want torch or tensorflow DL framework.  Default is "tf"
dqn_config.framework(framework="torch")

# Set the log level to DEBUG, INFO, WARN, or ERROR 
dqn_config.debugging(seed=415, log_level="ERROR")

# Setup evaluation
dqn_config.evaluation(
    
    # Minimum number of training iterations between evaluations.
    # Evaluations are blocking operations (if evaluation_parallel_to_training=False) 
    # set `evaluation_interval` larger for faster runtime.
    evaluation_interval=10, 

    # Minimum number of evaluation iterations.
    # If using multiple evaluation workers, we will run at least 
    # this many episodes * num_evalworkers total.
    evaluation_duration=5,      

    # Number of parallel evaluation workers. 
    # Zero by default, which means evaluation will run on the training resources. 
    # If you increase this, it will increase total Ray resource usage
    # since evaluation workers are created separately from rollout workers 
    # Note: these show up on Ray Dashboard as extra "RolloutWorker"s
    evaluation_num_workers=7,  #0 for Colab

    # Use the parallel evaluation workers in parallel with training workers
    evaluation_parallel_to_training=True,  #False for Colab
    
    evaluation_config = dict(
        # Explicitly set "explore"=False to override default True
        # Best practice value is False unless environment is stochastic
        explore=False,
        
        # Number of parallel Training workers
        # Override the num_workers from the training config 
        # Note: DQN only allows 1 Trainer worker, see documentation
        num_workers=1,  #any number here will be reset = 1 for DQN
    ),
)

# # Override default training parameters
# dqn_config.training(target_network_update_freq=5000, 
#                     model=dict("fcnet_hiddens" : [32, 32])
#                    )

# Setup sampling rollout workers for streaming the data 
dqn_config.rollouts(
    num_rollout_workers=7,  #1 for Colab
    
    # for small environments this can be >1 based on size of your processor
    num_envs_per_worker=1,)

print(f"Config type: {type(dqn_config)}")

# Use the config object's `build()` method for instantiating
# an RLlib Algorithm instance that we can then train.
# Note if using Tune, don't need algo object, but this is still a good debugging step.
dqn_algo = dqn_config.build()
print(f"Algorithm type: {type(dqn_algo)}")

print()
print("DQN MODEL ARCHITECTURE:")
# print(result['config']['model'])
# # tf print keras model summary
# print(dqn_algo.get_policy().model.base_model.summary())
# # torch
# from torchinfo import summary
# summary(dqn_algo.get_policy().model)


2023-12-02 12:02:14,403	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!


Config type: <class 'ray.rllib.algorithms.dqn.dqn.DQNConfig'>


/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and m

RaySystemError: System error: No module named 'cyborg_env'
traceback: Traceback (most recent call last):
  File "/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/ray/_private/serialization.py", line 404, in deserialize_objects
    obj = self._deserialize_object(data, metadata, object_ref)
  File "/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/ray/_private/serialization.py", line 270, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata_fields)
  File "/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/ray/_private/serialization.py", line 225, in _deserialize_msgpack_data
    python_objects = self._deserialize_pickle5_data(pickle5_data)
  File "/home/zhx/.conda/envs/academy/lib/python3.8/site-packages/ray/_private/serialization.py", line 215, in _deserialize_pickle5_data
    obj = pickle.loads(in_band)
ModuleNotFoundError: No module named 'cyborg_env'


In [ ]:
import sys
sys.path.append('/home/zhx/word/work/DriverOrderOfflineRL/cage-challenge-1/CybORG')

import ray
import numpy as np
import torch
import gymnasium as gym
from gymnasium import spaces
from gymnasium.wrappers import EnvCompatibility

from typing import Optional, Tuple, Union
from typing import Any, Dict, Optional, Protocol, runtime_checkable
from gymnasium.core import ObsType

import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Shared.Actions import *
from CybORG.Agents import RedMeanderAgent, B_lineAgent
from CybORG.Agents.Wrappers import *
path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'

# seed
np.random.seed(1)
torch.manual_seed(1)

# env
CYBORG = CybORG(path,'sim', agents={'Red': RedMeanderAgent})
class Wrapper(gym.Env):
    def __init__(self, env:ChallengeWrapper) -> None:
        self.env = env
        self.action_space = spaces.Discrete(54)
        self.observation_space = spaces.Box(low=-1.0, high=3.0, shape=((52,)), dtype=np.float32)
    
    def step(self, action=None):
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, info

    def reset(self):
        return self.env.reset()

class CyborgEnv(gym.Env):
    def __init__(self, config=None) -> None:
        self.env = EnvCompatibility(Wrapper(ChallengeWrapper(env=CybORG(path,'sim', agents={'Red': RedMeanderAgent}), agent_name="Blue", max_steps=100)))
        self.action_space = spaces.Discrete(54)
        self.observation_space = spaces.Box(low=-1.0, high=3.0, shape=((52,)), dtype=np.float32)

    def step(self, action: Any) -> Tuple[Any, float, bool, bool, Dict]:
        return self.env.step(action=action)

    def reset(
        self, seed: Optional[int] = None, options: Optional[dict] = None
    ) -> Tuple[ObsType, dict]:
        return self.env.reset(seed=seed, options=options)
